# Cats vs. Dogs - CNN Image Classifier

This notebook is my solution to the image classification coding challenge from [FreeCodeCamp "Machine Learning with Python"](https://www.freecodecamp.org/learn/machine-learning-with-python) course.

Here is my attempt to solve this assignment using Tensorflow and Keras.

## The Assignment

_"For this challenge, you will complete the code to classify images of dogs and cats. You will use TensorFlow 2.0 and Keras to create a convolutional neural network that correctly classifies images of cats and dogs at least 63% of the time. (Extra credit if you get it to 70% accuracy!)_

_Some of the code is given to you but some code you must fill in to complete this challenge. Read the instruction in each text cell so you will know what you have to do in each code cell._

_The first code cell imports the required libraries. The second code cell downloads the data and sets key variables. The third cell is the first place you will write your own code._

_The structure of the dataset files that are downloaded looks like this (You will notice that the test directory has no subdirectories and the images are not labeled):_

```
cats_and_dogs
|__ train:
    |______ cats: [cat.0.jpg, cat.1.jpg ...]
    |______ dogs: [dog.0.jpg, dog.1.jpg ...]
|__ validation:
    |______ cats: [cat.2000.jpg, cat.2001.jpg ...]
    |______ dogs: [dog.2000.jpg, dog.2001.jpg ...]
|__ test: [1.jpg, 2.jpg ...]
```

_You can tweak epochs and batch size if you like, but it is not required."_

## Building the Dataframes

In this section, we download the provided files and build the dataframes for training and testing de model. The provided data is already split in train/validation/test folders and each one has subfolders for cats and dogs: the two categories we have to work with.

Since the course is a bit outdated, it suggests you work with Tensorflow's __ImageDataGenerator__. Since this method is deprecated and not recommended anymore, we will use Keras' __image_dataset_from_directory__ for loading the images in the form of _tf.data.Dataframe_.

In [8]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import shutil
from zipfile import ZipFile

import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Download project files
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip

# Extract zip and clean unwanted files
with ZipFile("cats_and_dogs.zip", "r") as zObject:
    zObject.extractall()

# Remove the remaining files we don't need
os.remove("cats_and_dogs.zip")
shutil.rmtree("__MACOSX")
PATH = "cats_and_dogs"

# Save directories for training, validation and test folders.
# Train/validation split is already done in the folder structure.
train_dir = os.path.join(PATH, "train")
validation_dir = os.path.join(PATH, "validation")
test_dir = os.path.join(PATH, "test")

--2024-01-05 17:32:22--  https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70702765 (67M) [application/zip]
Saving to: 'cats_and_dogs.zip'

     0K .......... .......... .......... .......... ..........  0% 9,06M 7s
    50K .......... .......... .......... .......... ..........  0%  101M 4s
   100K .......... .......... .......... .......... ..........  0%  163M 3s
   150K .......... .......... .......... .......... ..........  0%  153M 2s
   200K .......... .......... .......... .......... ..........  0% 12,5M 3s
   250K .......... .......... .......... .......... ..........  0% 6,60M 4s
   300K .......... .......... .......... .......... ..........  0%  104M 4s
   350K .......... .......... .......... .......... .........

In [10]:
# Variables for pre-processing and training.
BATCH_SIZE = 32
EPOCHS = 30
IMG_HEIGHT = 250
IMG_WIDTH = 250

We build three different dataframes: one for training the model and one for validating each epoch results. The third one only has a few samples and is meant for manual testing. The images will be packed in three _tf.data.Dataframe_ objects, with batching and automatic labels based in the folder structure.

In [11]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels="inferred",
    label_mode="binary",
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode="rgb",
    shuffle=True
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=validation_dir,
    labels="inferred",
    label_mode="binary",
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode="rgb",
    shuffle=True
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    #batch_size=None,
    color_mode="rgb",
    image_size=(IMG_HEIGHT, IMG_WIDTH)
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 50 files belonging to 1 classes.


## Building the Model (Rehacer este markdown)

To perfom image classification of colour images, we have to build a __Convolutional Neural Network__. In this case, I made a simple one with the following architecture:

1. Rescaling layer
2. Multiple Data Augmentation layers
3. Multiple Conv2D -> MaxPool2D layers
4. Densely Connected classification layer

The final layer is a 2-neuron Dense layer that output one of the two possible categories: dog or cat.

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.Rescaling(scale=1./255),
    
    tf.keras.layers.RandomFlip(mode="horizontal"),
    tf.keras.layers.RandomRotation(factor=0.3),
    tf.keras.layers.RandomZoom(height_factor=(0.0, 0.4)),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 250, 250, 3)       0         
                                                                 
 random_flip_3 (RandomFlip)  (None, 250, 250, 3)       0         
                                                                 
 random_rotation_3 (RandomR  (None, 250, 250, 3)       0         
 otation)                                                        
                                                                 
 random_zoom_3 (RandomZoom)  (None, 250, 250, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 124, 124, 32)      0         
 ng2D)                                                

In [ ]:
model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, shuffle=True)